In [1]:
import time
import requests
import yaml
import logging
from collections import defaultdict

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to parse the YAML configuration file
def parse_yaml(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

# Function to perform a health check on the endpoint
def health_check(endpoint):
    try:
        if endpoint.get('method', 'GET') == 'GET':
            response = requests.get(endpoint['url'], headers=endpoint.get('headers', {}), timeout=5)
        else:  # POST method
            response = requests.post(endpoint['url'], headers=endpoint.get('headers', {}), json=endpoint.get('body', {}), timeout=5)

        latency = response.elapsed.total_seconds()
        return response.status_code in range(200, 300) and latency < 0.5, latency  # Return UP status and latency
    except requests.RequestException as e:
        return False, None  # Endpoint is DOWN

# Main function to run the health checks
def run_health_checks(file_path):
    endpoints = parse_yaml(file_path)
    results = defaultdict(lambda: {'up': 0, 'total': 0})

    try:
        while True:
            for endpoint in endpoints:
                is_up, latency = health_check(endpoint)
                domain = endpoint['url'].split('//')[-1].split('/')[0]
                results[domain]['total'] += 1
                if is_up:
                    results[domain]['up'] += 1

                status = "UP" if is_up else "DOWN"
                log_message = f"{endpoint['name']} is {status} with latency {latency if latency else 'N/A'}"
                logging.info(log_message)
                print(log_message)  # Print the status

            # Log and print the availability percentage
            for domain, result in results.items():
                availability_percentage = (result['up'] / result['total']) * 100
                availability_message = f"{domain} has {availability_percentage:.2f}% availability percentage"
                logging.info(availability_message)
                print(availability_message)  # Print the availability percentage

            time.sleep(15)

    except KeyboardInterrupt:
        exit_message = "Program exited by user"
        logging.info(exit_message)
        print(exit_message)  # Print the exit message

if __name__ == "__main__":
   config_file_path = '/content/sample.yaml'

   run_health_checks(config_file_path)


fetch.com index page is UP with latency 0.277085
fetch.com careers page is UP with latency 0.333264
fetch.com some post endpoint is DOWN with latency 0.111657
www.fetchrewards.com index page is UP with latency 0.109896
fetch.com has 66.67% availability percentage
www.fetchrewards.com has 100.00% availability percentage
fetch.com index page is UP with latency 0.175324
fetch.com careers page is UP with latency 0.177389
fetch.com some post endpoint is DOWN with latency 0.166142
www.fetchrewards.com index page is UP with latency 0.162741
fetch.com has 66.67% availability percentage
www.fetchrewards.com has 100.00% availability percentage
fetch.com index page is UP with latency 0.155398
fetch.com careers page is UP with latency 0.110862
fetch.com some post endpoint is DOWN with latency 0.110471
www.fetchrewards.com index page is UP with latency 0.113722
fetch.com has 66.67% availability percentage
www.fetchrewards.com has 100.00% availability percentage
fetch.com index page is UP with laten